# Create image log mesh using Open3D Poisson Surface Reconstruction Algorithm

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

from test_dlis_load_function import dlis_loader
from test_raster_image import raster_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load wellbore image data

In [2]:
file = '.\\data\\58-32_processed_image\\DLIS_XML_ProcessedImages\\University_of_Utah_MU-ESW1_FMI-HD_7390-7527ft_Run3.dlis'
#file = '.\\data\\58-32_FMI_DLIS_XML\\University_of_Utah_MU_ESW1_FMI_HD_2226_7550ft_Run1.dlis'
#file = '.\\data\\58-32_FMI_DLIS_XML\\University_of_Utah_MU_ESW1_FMI_HD_7440_7550ft_Run2.dlis'

curves_frame = dlis_loader(file)
tdep = curves_frame['TDEP']
P1NO_FBST_S = curves_frame['P1NO_FBST_S']
RB_FBST_S = curves_frame['RB_FBST_S']
HAZIM_S = curves_frame['HAZIM_S']
DEVIM_S = curves_frame['DEVIM_S']
C1_S = curves_frame['C1_S']
C2_S = curves_frame['C2_S']
BS = curves_frame['BS']
fmi_dyn = curves_frame['FMI_DYN']
fmi_dyn[fmi_dyn==-9999] = np.nan
fmi_stat = curves_frame['FMI_STAT']
fmi_stat[fmi_stat==-9999] = np.nan

fmi_raster = raster_image(fmi_stat, 'stat')

In [ ]:
fmi_raster = np.array([i for i in fmi_raster[15100:-1,:,:]])
BS = np.array([i for i in BS[15100:-1]])
HAZIM_S = np.array([i for i in HAZIM_S[15100:-1]])
DEVIM_S = np.array([i for i in DEVIM_S[15100:-1]])
P1NO_FBST_S = np.array([i for i in P1NO_FBST_S[15100:-1]])
RB_FBST_S = np.array([i for i in RB_FBST_S[15100:-1]])

In [11]:
dev_rad = np.pi*(90-DEVIM_S)/180
azi_rad = np.pi*(HAZIM_S)/180

dl_rad = [np.arccos(np.cos(dev_rad[z]-dev_rad[z-1])-np.sin(dev_rad[z-1])*np.sin(dev_rad[z])*(1-np.cos(azi_rad[z]-azi_rad[z-1]))) for z in range(dev_rad.shape[0])]
dl_rad[0] = dl_rad[1]
rf = [np.tan(dl_rad/2)/(dl_rad/2) if dl_rad!=0.0 else np.nan for dl_rad in dl_rad]
steps = [s for s in dl_rad if dl_rad!=0.0]
dev_x = steps/2*(SIN(I18)*SIN(J18)+SIN(I19)*SIN(J19))*M19

In [ ]:
np.arccos(np.cos(0)-np.sin(0.5236)*np.sin(0.47124)*(1-np.cos(0.0)))

In [12]:
c = 1.5
r = BS*0.5
fmi_xyz = np.array([[[np.cos(i*0.0174532925)*(r[z]+(fmi_raster[z,i,0]*c)-(fmi_raster[z,fmi_raster[z,:,0]!=0,0]*c).mean() if fmi_raster[z,i,0]!=0.0 else r[z]), np.sin(i*0.0174532925)*(r[z]+(fmi_raster[z,i,0]*c)-(fmi_raster[z,fmi_raster[z,:,0]!=0,0]*c).mean() if fmi_raster[z,i,0]!=0.0 else r[z]), z/10] for i in range(fmi_raster.shape[1])] for z in range(fmi_raster.shape[0])])
fmi_xyz_n = np.array([[[1*np.cos(i*0.0174532925)/(r[z]), 1*np.sin(i*0.0174532925)/(r[z]), 0.0] for i in range(fmi_raster.shape[1])] for z in range(fmi_raster.shape[0])])

In [13]:
fmi_raster = fmi_raster.reshape(-1, fmi_raster.shape[-1])
fmi_xyz = fmi_xyz.reshape(-1, fmi_xyz.shape[-1])
fmi_xyz_n = fmi_xyz_n.reshape(-1, fmi_xyz_n.shape[-1])

In [14]:
todrop = [row for row in range(fmi_raster.shape[0]) if fmi_raster[row].all()==0]     
fmi_raster = np.delete(fmi_raster, todrop, axis=0)
fmi_xyz = np.delete(fmi_xyz, todrop, axis=0)
fmi_xyz_n = np.delete(fmi_xyz_n, todrop, axis=0)

## Preprocessing by creating point cloud data from the wellbore image arrays

In [15]:
# Pass xyz, RBG and normals to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fmi_xyz)
pcd.colors = o3d.utility.Vector3dVector(fmi_raster[:,:3])
pcd.normals = o3d.utility.Vector3dVector(fmi_xyz_n)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

## Poisson surface reconstruction

In [19]:
# runing Poisson surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=14)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [20]:
#  visualize mesh vertrex densities
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')((densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh], mesh_show_back_face=True)

In [21]:
# removing low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.05)
mesh.remove_vertices_by_mask(vertices_to_remove)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("mesh.ply", mesh)

[Open3D WARNING] Write Ply clamped color value to valid range


True